# Configuracion Principal

In [ ]:
%pip install pandas openpyxl litellm langchain python-dotenv

In [29]:
import os
from time import sleep

import pandas as pd
import litellm
litellm.drop_params = True
from litellm import completion
from dotenv import load_dotenv, find_dotenv
from typing import List, Tuple, Dict
from tkinter import Tk, messagebox

def popup(message, title=None):
    root = Tk()
    root.withdraw()
    root.wm_attributes("-topmost", 1)
    messagebox.showinfo(title, message, parent=root, default="ok")
    root.destroy()

# Debe de haber en la raiz del proyecto un archivo '.env' con las api key, Ej:
# ------- .env -------------------
# GROQ_API_KEY = <API KEY GROQ>
# -------------------------------

load_dotenv(find_dotenv()) # Busca y carga el archivo .env
# Acceder a la API key
groq_api_key = os.getenv("GROQ_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")
together_api_key = os.getenv("TOGETHERAI_API_KEY")


### Funciones de Utilidades

In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox
from pathlib import Path

def seleccionar_archivo_xlsx(msg:str = "Seleccionar archivos xlsx")-> list[Path]:
    # Crear la ventana de selección de archivo
    root = tk.Tk()
    root.withdraw()
    root.overrideredirect(True)
    root.geometry('0x0+0+0')
    root.deiconify()
    root.lift()
    root.focus_force()
    root.attributes('-alpha', 0.0)
    # Window mantain in front of everything
    root.attributes('-topmost', True)
    
    file_paths = filedialog.askopenfilenames(
        parent=root,
        title=msg,
        filetypes=[("Excel files", "*.xlsx *.xls")]
    )
    root.destroy()
    if len(file_paths) == 0:
        raise Exception("No se ha selecciondo un archivo excel")
    return list(Path(file) for file in file_paths)

def seleccionar_carpeta(msg:str = "Seleccionar carpeta") -> Path:
    # Crear la ventana de selección de carpeta
    root = tk.Tk()
    root.withdraw()
    root.overrideredirect(True)
    root.geometry('0x0+0+0')
    root.deiconify()
    root.lift()
    root.focus_force()
    root.attributes('-alpha', 0.0)
    # Mantener la ventana en frente de todo
    root.attributes('-topmost', True)
    
    carpeta_path = filedialog.askdirectory(
        parent=root,
        title=msg
    )
    root.destroy()
    if (carpeta_path is None) or (carpeta_path == ""):
        raise Exception("No se ha selecciondo una carpeta")
    return Path(carpeta_path)


## Cargar Archivo con Preguntas
Columnas que debe tener:
- numero: numero enteroque representa el número de la pregunta
- pregunta: texto que representa el enunciado de la pregunta
- a: texto que representa la respuesta de a 
- b: texto que representa la respuesta de b 
- c: texto que representa la respuesta de c 
- d: texto que representa la respuesta de d 
- respuesta: letra que indica cual es la alternativa correcta de la pregunta


In [4]:
# Seleccionar el archivo xlsx con las preguntas
ruta_archivo = seleccionar_archivo_xlsx(msg="Seleccione el archivo que contiene las preguntas")

df = pd.read_excel(ruta_archivo[0])

if set(df.columns).issubset(["numero", "pregunta", "a", "b", "c", "d", "respuesta"]):
    raise Exception("Verifique que estan las columnas necesarias en el archivo xlsx seleccionado")
else:
    print("(OK) El archivo cumple con la estructura requerida y ha sido cargado correctamente")

(OK) El archivo cumple con la estructura requerida y ha sido cargado correctamente


In [11]:
df.head(3)

,tipo_prueba,numero,contenido_especial,pregunta,categoria,a,b,c,d,respuesta
0,en_paes_2024_matematicas_1,1,NaN,¿Cuál es el resultado de 3 - (-1)(-1-5)?,numeros,-1,-3,-12,-24,b
1,en_paes_2024_matematicas_1,3,NaN,Si al quíntuplo de -10 se le resta el triple d...,numeros,-86,-14,2,34,b
2,en_paes_2024_matematicas_1,7,NaN,Una persona realiza un viaje al exterior y lle...,numeros,$254000,$244000,$248000,$246000,d


# Función LLamadas a API

In [30]:
import os
from time import time, sleep
from random import randint
import json

import pandas as pd
import litellm
litellm.drop_params = True
from litellm import completion
from langchain import PromptTemplate

def get_api_response(
        model:str,
        api_key:str,
        base_url:str | None,
        df: pd.DataFrame,
        system_prompt: str,
        user_prompt_template:PromptTemplate,
        parameters:dict,
        sleep_per_question:float=0,
        ) -> pd.DataFrame:
    """
    This function interacts with an API to get responses based on a set of questions and their parameters.
    
    Args:
        model (str): The model name (litellm) to be used for the API.
        api_key (str): The API key for authentication.
        base_url (str | None): The base URL for the API.
        df (pd.DataFrame): A dataframe containing the questions to be asked.
        system_prompt (str): The system-level prompt for the API.
        user_prompt_template (langchain.PromptTemplate): A template for the user-level prompts.
        parameters (dict): A dictionary of parameters for the API request.
        sleep_per_question (float): A float that represents the time to sleep per question
    Returns:
        pd.DataFrame: A dataframe containing the API responses and related metadata.
    """

    responses:pd.DataFrame = pd.DataFrame(
        {
        "model": [],
        "exec_time": [],
        "params": [],
        "question_num": [],
        "question_statement": [],
        "sys_content": [],
        "user_content": [],
        "llm_response": [],
        "usage": []
        }
    )
    # print(config)
    df_questions:pd.DataFrame = df
    system_content:str = system_prompt
    parameters = parameters

    for idx, row in df_questions.iterrows():
        print(f"Pregunta {row['numero']}...")
        if "e" in row.columns:
            user_content:str = user_prompt_template.format(
                question=row["pregunta"],
                a = row["a"],
                b = row["b"],
                c = row["c"],
                d = row["d"],
                e = "e)" + str(row["e"]),
            )
        else:
            user_content:str = user_prompt_template.format(
                question=row["pregunta"],
                a = row["a"],
                b = row["b"],
                c = row["c"],
                d = row["d"],
                e = "",
            ).replace("\n\n", "\n")

        retry_counter = 0
        retry_time = 14
        while 1:
            try:
                retry_counter+= 1
                start_time:float = time()
                if system_content != "":
                    messages = [
                        {"role": "system", "content": system_content},
                        {"role": "user", "content": user_content},
                    ]
                else:
                    messages = [
                        {"role": "user", "content": user_content},
                    ]
                llm_response = completion(
                    model = model,
                    api_key= api_key,
                    base_url= base_url,
                    messages = messages,
                    temperature = parameters["temperature"],
                    max_tokens = parameters["max_tokens"],
                    frequency_penalty = parameters["frequency_penalty"],
                    stream = parameters["stream"],
                    top_p = parameters["top_p"],
                    )
                end_time:float = time()
                exec_time:float = round(end_time - start_time, 4)
                sleep(randint(1,7)/10)
                break
            except Exception as e:
                if retry_counter >= 5:
                    print(f"\n\nERROR FATAL AL INTENTAR CONECTARSE CON LA API \n{e.args}\n\n")
                    # popup("ERROR FATAL AL RE-INTENTAR CONECTAR CON LA API", "ERROR CONECCION API")
                    return responses
                else:
                    print(f"Error inesperado: {e.args}\nReintentado en {retry_time * retry_counter}...")
                    start_time, end_time = 0
                    del start_time, end_time
                    sleep(retry_time*retry_counter)
                    print(F"> Reintento N° {retry_counter}...")
            #END TRY
        #END WHILE

        llm_usage:dict= {   
            "completion_tokens": llm_response.usage.completion_tokens,
            "prompt_tokens": llm_response.usage.prompt_tokens,
            "total_tokens": llm_response.usage.total_tokens
        }

        response = {
            "model": model,
            "exec_time": exec_time,
            "params": json.dumps(parameters),
            "question_num": row["numero"],
            "question_statement": row["pregunta"],
            "sys_content": system_content,
            "user_content": user_content,
            "llm_response": llm_response.choices[0].message.content,
            "usage": json.dumps(llm_usage),
        }

        responses:pd.DataFrame = pd.concat([responses, pd.DataFrame([response])], ignore_index=True)
        del start_time, end_time, exec_time
        # print(f" > API response:\n > {llm_response.choices[0].message.content}")
        print("- Done -")
        if type(sleep_per_question) == list:
            sleep(randint(sleep_per_question[0],sleep_per_question[1]))
        elif type(sleep_per_question) == float:
            sleep(sleep_per_question)
    #END FOR

    return responses

# Definicion de Prompts

In [32]:
from langchain.prompts import PromptTemplate
system = "" # NOTE: Por defecto dejar en "" si no se quiere usar un prompt de systema
prompt_0 = PromptTemplate.from_template(
"""{question}
Alternativas:
a) {a}
b) {b}
c) {c}
d) {d}
{e}
""")

prompt_1 = PromptTemplate.from_template(
"""{question}
Alternativas:
a) {a}
b) {b}
c) {c}
d) {d}
{e}
Piensa paso a paso antes de responder
""")

prompt_2 = PromptTemplate.from_template(
"""Necesito tu ayuda para resolver el siguiente problema, primero te daré 5 ejemplos de preguntas y su respuesta esperada:

Ejemplo 1: De la expresión (a+b)^2 + 5 = 0. ¿Cuál de las siguientes opciones es equivalente a esta expresión?
Alternativas:
a) a^2 + 2ab + b^2 = 5
b) a^2 + b^2 + 5 = 0
c) a^2 + 2ab + b^2 = -5
d) a+b = -5
Respuesta: letra c), la solución es (a+b)^2 + 5 = a^2 + 2ab + b^2 + 5, restando 5 de cada lado tenemos a^2 + 2ab + b^2 = -5.

Ejemplo 2: Si Pedro tenía 2 perros y Juan le da uno más, ¿cuántos perros tiene Pedro?
Alternativas:
a) 0
b) 1
c) 2
d) 3
Respuesta: letra d), pedro tiene 2 perros y recibe uno más, entonces tiene 3 perros.

Ejemplo 3: Dan lanza una moneda 3 veces, ¿cuál es la probabilidad de obtener 3 caras seguidas?
Alternativas:
a) (1/2)^3
b) (1/2)
c) (1/2)^2
d) (1/2)^4
Respuesta: letra a), la probabilidad de obtener cara en 1 lanzamiento es 1/2, sí lanzamos 3 veces y queremos 3 caras, entonces la probabilidad es: (1/2)^3.

Ejemplo 4: Un coche rojo comienza un viaje en la posición (0,0) km, viaja 1/2 km hacia el norte y luego 2 km hacia el este, ¿cuál es su posición final?
Alternativas:
a) (1,0)
b) (1.5,-2)
c) (1.5,2)
d) (2,1.5)
Respuesta: letra c), Comenzamos en (0,0) y viajamos (1/2) = 1.5 km al norte llegando a (1.5,0) km, luego vamos 2 km al este, llegando a (1.5,2) km.

Ejemplo 5: Sea x la cantidad de manzanas que compramos, ademas por cada manzana se pagan $500 más un costo fijo de $25, ¿cuál expresión permite calcular el costo final?
Alternativas:
a) $ 500x + 25x
b) $ (500+25)x
c) $ 25x + 500
d) $ 500x + 25
Respuesta: letra d), por cada manzana pagamos $500, teniendo 500*x más un costo fijo de $25, entonces se tiene 500x + 25.
Ahora resuelve esta pregunta: {question}
Alternativas:
a) {a}
b) {b}
c) {c}
d) {d}
{e}
""")

prompt_3 = PromptTemplate.from_template(
"""Necesito tu ayuda para resolver el siguiente problema, primero te daré 5 ejemplos de preguntas y su respuesta esperada:

Ejemplo 1: De la expresión (a+b)^2 + 5 = 0. ¿Cuál de las siguientes opciones es equivalente a esta expresión?
Alternativas:
a) a^2 + 2ab + b^2 = 5
b) a^2 + b^2 + 5 = 0
c) a^2 + 2ab + b^2 = -5
d) a+b = -5
Respuesta: letra c), la solución es (a+b)^2 + 5 = a^2 + 2ab + b^2 + 5, restando 5 de cada lado tenemos a^2 + 2ab + b^2 = -5.

Ejemplo 2: Si Pedro tenía 2 perros y Juan le da uno más, ¿cuántos perros tiene Pedro?
Alternativas:
a) 0
b) 1
c) 2
d) 3
Respuesta: letra d), pedro tiene 2 perros y recibe uno más, entonces tiene 3 perros.

Ejemplo 3: Dan lanza una moneda 3 veces, ¿cuál es la probabilidad de obtener 3 caras seguidas?
Alternativas:
a) (1/2)^3
b) (1/2)
c) (1/2)^2
d) (1/2)^4
Respuesta: letra a), la probabilidad de obtener cara en 1 lanzamiento es 1/2, sí lanzamos 3 veces y queremos 3 caras, entonces la probabilidad es: (1/2)^3.

Ejemplo 4: Un coche rojo comienza un viaje en la posición (0,0) km, viaja 1/2 km hacia el norte y luego 2 km hacia el este, ¿cuál es su posición final?
Alternativas:
a) (1,0)
b) (1.5,-2)
c) (1.5,2)
d) (2,1.5)
Respuesta: letra c), Comenzamos en (0,0) y viajamos (1/2) = 1.5 km al norte llegando a (1.5,0) km, luego vamos 2 km al este, llegando a (1.5,2) km.

Ejemplo 5: Sea x la cantidad de manzanas que compramos, ademas por cada manzana se pagan $500 más un costo fijo de $25, ¿cuál expresión permite calcular el costo final?
Alternativas:
a) $ 500x + 25x
b) $ (500+25)x
c) $ 25x + 500
d) $ 500x + 25
Respuesta: letra d), por cada manzana pagamos $500, teniendo 500*x más un costo fijo de $25, entonces se tiene 500x + 25.
Ahora resuelve esta pregunta: {question}
Alternativas:
a) {a}
b) {b}
c) {c}
d) {d}
{e}
Piensa paso a paso antes de responder
""")



### Ejemplo de Uso

In [33]:
mensaje = prompt_1.format(
    question="Pregunta de Prueba 1",
    a="Alternativa <a>",
    b="Alternativa <b>",
    c="Alternativa <c>",
    d="Alternativa <d>",
    e=""
    ).replace("\n\n", "\n")
print(mensaje)
print("---------------")
mensaje = prompt_1.format(
    question="Pregunta de Prueba 2",
    a="Alternativa <a>",
    b="Alternativa <b>",
    c="Alternativa <c>",
    d="Alternativa <d>",
    e="e) " + str("Alternativa <e>")
    ).replace("\n\n", "\n")
print(mensaje)

Pregunta de Prueba 1
Alternativas:
a) Alternativa <a>
b) Alternativa <b>
c) Alternativa <c>
d) Alternativa <d>
Piensa paso a paso antes de responder

---------------
Pregunta de Prueba 2
Alternativas:
a) Alternativa <a>
b) Alternativa <b>
c) Alternativa <c>
d) Alternativa <d>
e) Alternativa <e>
Piensa paso a paso antes de responder



## Uso de Función de llamada de API

A continuación se debe de seleccionar la carpeta donde se guardaran los archivos resultados de las pruebas

NOTA: LA CARPETA DEBE TENER EL NOMBRE  "prompt..." AL INICIO

In [ ]:
ruta_para_guardar_archivos = seleccionar_carpeta(msg="Selecciona la carpeta donde se guardaran los resultados de las llamadas")
print("Los archivos generados se guardaran en:\n",ruta_para_guardar_archivos)


### Código que llama a las APIs

In [13]:

base_url = "" #Por defecto es "" si no se usa un LLM de manera local
# -------------- PARAMETROS -------------------

# MODELO:
modelo:str = "groq/llama3-70b-8192"
# modelo:str = "groq/mixtral-8x7b-32768"
# modelo:str = "openai/gpt-3.5-turbo-0125"
# modelo:str = "together_ai/mistralai/Mixtral-8x22B-Instruct-v0.1"
# modelo:str = "openai/lmstudio-community/Phi-3-mini-4k-instruct-GGUF"
#! base_url = "http://localhost:1224/v1" # En caso de usar LM Studio configurar acorde

# API KEY:
api_key = groq_api_key

# PARAMETROS:
parametros:dict = {
    "temperature": 0,
    "max_tokens": 2_700,
    "frequency_penalty": 0,
    "stream": False,
    "top_p": 1.0,
}

# CONFIGURACIÓN GUARDADO ARCHIVOS
ITERACIONES:int = 1 
NUM_INICIAL_GUARDAR:int = 1 #Número con el que se empezara a guardar el archivo ej: "result_1"
SLEEP_PER_QUESTION:list[float] = [2,5] #Numero random entre estos valores que se descansara entre pregunta
MENSAJE_AL_TERMINAR:bool = True # True si quiere un popup en pantalla al terminar de ejecutar codigo
# -------------------------------------------------

base_url = None if base_url == "" else base_url
_lista_aux_errores:list[tuple] = []
for i in range(ITERACIONES):
    print("### Iteracion: ", i+1, " ###")

    responses = get_api_response(
        model = modelo,
        api_key = api_key,
        base_url = base_url,
        df = df,
        system_prompt = system,
        user_prompt_template = prompt_1,
        parameters = parametros,
        sleep_per_question = SLEEP_PER_QUESTION,
    )

    nombre_archivo = f"resultado_{NUM_INICIAL_GUARDAR}.xlsx"
    ruta_archivo_completa = os.path.join(ruta_para_guardar_archivos, nombre_archivo)

    if os.path.exists(ruta_archivo_completa):
        print(f"(X.X) El archivo {nombre_archivo} ya existe en la ruta. No se guardará para evitar sobreescribirlo.")
        _lista_aux_errores.append(("El archivo ya existe en la ruta seleccionada",responses))
    else:
        try:
            responses.to_excel(ruta_archivo_completa)
            print(f"Archivo guardado exitosamente en iteración: {i+1}")
        except Exception as e:
            print(f"(X.X) Error al intentar guardar archivo en iteración: {i+1}: {e}")
            _lista_aux_errores.append(("No se pudo guardar el archivo",responses))
    
    NUM_INICIAL_GUARDAR += 1

if MENSAJE_AL_TERMINAR:
    popup(message= f"Se han terminado de ejecutar las {ITERACIONES} iteraciones",title="Proceso terminado")
if len(_lista_aux_errores)>0:
    print("----------------------------------------\n(O.O) Importante verificar la variable '_lista_aux_errores' ya que han sido guardados\narchivos en esta lista que no se han podido guardar correctamente\n----------------------------------------")


Iteracion:  1
Pregunta 1...
- Done -
Pregunta 3...
- Done -
Pregunta 7...
- Done -
Pregunta 8...
- Done -
Pregunta 14...
- Done -
Pregunta 15...
- Done -
Pregunta 17...
- Done -
Pregunta 19...
- Done -
Pregunta 22...


KeyboardInterrupt: 

# Análisis REGEX de resultados

In [34]:
class RegexAnalyzerFisica():
    def __init__(
            self, 
            files_paths:str=None, 
            tesis_path:str=None, 
            respuestas_correctas:list | None=None,
            guardar_registros:bool=True
            ) -> None:
        self.index:int = 0
        self.respuestas_correctas:list = respuestas_correctas
        self.guardar_registros:bool = guardar_registros

        # get files paths
        if files_paths == None:
            self.files_paths:list[Path] = self.get_files()
        else:
            self.files_paths:list[Path] = files_paths
        if tesis_path == None:
            self.tesis_path:Path = self.get_tesis_path()
        else:
            self.tesis_path:Path = tesis_path

        if self.tesis_path == None:
            raise Exception("Tesis path is None")
        elif len(self.files_paths) == 0:
            raise Exception("Files path is empty")
        elif len(self.files_paths) != 30:
            print("Amount of files loaded are not equal to 30 xlsx files")

        # get llm_answers per file
        for file_path in self.files_paths:
            if file_path == None:
                print("NONE FILE DETECTED AND SKIPPED")
                continue

            # We create llm_answers wich would have all the anwsers given by the model
            llm_answers:pd.DataFrame = pd.DataFrame(
                {
                    "llm_answer": []
                }
            )
            df:pd.DataFrame = pd.read_excel(file_path)
            # print("DF Head: ", df.head(2)) # !DEBBUG
            
            if not df.columns.equals(pd.Index([
                    "Unnamed: 0","model", "exec_time", "params", "question_num", "question_statement",
                    "sys_content", "user_content", "llm_response", "usage"
                    ])
                ):
                raise Exception(f"The dataframe doesnt match the desire structure\nFile_path: {file_path}")
            df.drop(columns=["Unnamed: 0"], inplace=True)

            if self.check_correct_column_in_df(df=df):
                print(f"> Correct column already created, dropping the column... {file_path}")
                df.drop(columns=['correct'], inplace=True) #NOTE: Eliminate the column...

            # We get all the 30 answers given by the llm
            for idx, row in df.iterrows():
                llm_answer:str = self.get_llm_answer(llm_response=row["llm_response"])

                new_record:dict = {
                    "llm_answer": llm_answer
                }
                llm_answers = pd.concat(
                    [llm_answers, pd.DataFrame([new_record])], 
                    ignore_index=True
                )
            #END FOR (iterrows)
            
            # check if the answer was correct
            #  > calculate accuracy
            llm_answers:pd.DataFrame = self.calculate_accuracy(llm_answers=llm_answers)
            print("\nFILE READED:\n", os.path.basename(file_path))
            print(f"   (0-0)< |Accuracy: {((llm_answers['correct'].sum())/30)*100}% |")
            try:
                df_data = df.copy()
                df_data = pd.concat([df_data, llm_answers], axis=1)
            except Exception as e:
                raise Exception(f"Error fatal con : {file_path}\nCuando se intento guardar los datos de 'llm_answers' y 'correct' en df\n{e.args}")

            # Save in tidy format
            #  > open & save file in tidy data format (path:.../RESULTADOS_TESIS/tidy_data_format/tesis_resulsts.xlsx)
            #     |model|temperature|prompt_id|file_id|question_number|correct:bool|llm_answer|completion_tokens|prompt_tokens|exec_time|
            df_tesis:pd.DataFrame = self.save_data_to_tesis_file(
                tesis_path = self.tesis_path, 
                df_data = df_data, 
                file_path = file_path
            )
            df_tesis.to_excel(self.tesis_path, index=False)
            print("Tesis Results Updated succesfully")
            print("Tesis Path: ", self.tesis_path)
        #END FOR (file_paths)
    def get_files(self) -> list[Path]:
        root = tk.Tk()
        root.withdraw()
        root.overrideredirect(True)
        root.geometry('0x0+0+0')
        root.deiconify()
        root.lift()
        root.focus_force()
        root.attributes('-alpha', 0.0)
        # Window mantain in front of everything
        root.attributes('-topmost', True)
        
        file_paths = filedialog.askopenfilenames(
            parent=root,
            title="Seleccionar archivos",
            filetypes=[("Excel files", "*.xlsx *.xls")]
        )
        root.destroy()

        # Convierte returns Path instances in a list
        return [Path(file_path) for file_path in file_paths] if file_paths else []
    
    def get_tesis_path(self) -> Path:
        root = tk.Tk()
        root.withdraw()  # Oculta la ventana principal de tkinter
        msg = "A continuación debe seleccionar el archivo xlsx que será utilizado para guardar los datos en formato Tidy Data\nColumnas que debe tener:\n"
        msg += '["model", "temperature", "prompt_id", "file_id", "question_number","correct", "llm_answer", "completion_tokens", "prompt_tokens", "exec_time"]'
        popup(msg)
        # Abre el cuadro de diálogo para seleccionar un archivo
        file_path = filedialog.askopenfilename(
            title="Seleccionar archivo de tesis",
            filetypes=[("Archivos Excel", "*.xlsx")]
        )
        del msg
        if file_path:
            return Path(file_path)
        else:
            return None     

    def check_correct_column_in_df(self, df:pd.DataFrame) -> bool:
        if "correct" in df.columns:
            return True
        else:
            return False 

    def any_filter_in_text(self, filters, text):
        return any(f in text for f in filters)

    def get_llm_answer(self, llm_response:str) -> str:
        text:str = llm_response
        text = text.lower()

        text = text.replace("\n\n","\n")
        text = text.replace("\n","")
        text = text.replace("es:","es ")
        text = text.replace(", por lo tanto,","")
        text = text.replace(" therefore","")

        text = text.replace(" **","")
        text = text.replace("  "," ")
        text = text.replace("`","")
        text = text.replace(":","")
        text = text.replace("esa)","a)")
        text = text.replace("esb)","b)")
        text = text.replace("esc)","c)")
        text = text.replace("esd)","d)")
        text = text.replace("(a)","a")
        text = text.replace("(b)","b")
        text = text.replace("(c)","c")
        text = text.replace("(d)","d")
        text = text.replace("opciona)","a)")
        text = text.replace("opcionb)","b)")
        text = text.replace("opcionc)","c)")
        text = text.replace("opcionc)","c)")
        text = text.replace("opciond)","d)")
        text = text.replace("opciond)","d)")
        text = text.replace("seriaa)","a)")
        text = text.replace("seriab)","b)")
        text = text.replace("seriac)","c)")
        text = text.replace("seriad)","d)")
        
        text = text.replace("á","a")
        text = text.replace("é","e")
        text = text.replace("í","i")
        text = text.replace("ó","o")
        text = text.replace("u","u")


        letter:str = "X"
        # ------------------- FILTERS -----------------------------
        none_filters = ["ninguna de las anteriores", "la respuesta no se encuentra", "respuesta correcta es no esta",
                        "como solo se puede elegir una opcion", "haya un error en", "hubo un error en la formulacion",
                        "haya habido un error", "no puedo seleccionar ninguna", "ninguna de ellas coincide con nuestro resultado", 
                        "no hay una opcion correcta única", "no hay una opcion correcta entre",
                        "verifique las opciones y vuel", "esta opcion no esta entre las alternativas", "no es posible seleccionar una sola opcion",
                        "ninguna opcion coincide exactamente", "puede haber un error", "verificar si la pregunta esta correctamente formulada",
                        "exista un error en las opciones", "pregunta esta mal formulada", "no hay una solucion exacta entre",
                        ]

        a_filters = [
            "la respuesta correcta es a)", "la respuesta correcta es: a)", "puedo concluir que la opcion a) es la inferencia consistente",
            "respuesta correcta es: a)", "la hipótesis que guio el experimento es: a)", "el objetivo de investigación pertinente es: a)",
            "respuesta correcta es la opcion a)", "la opcion correcta es: a)", "la respuesta más probable es la a)",
            "la opcion a) es la inferencia consistente", "la opcion a) es la que permite comprobar la relación",
            "la pregunta que se ajusta a este experimento es: a)", "la respuesta más precisa es: a)",
            "lo que se puede hacer mediante la opcion a).",
            "en conclusion, la opcion a) es la que permite comprobar la relacion", "la opcion a) es la mas adecuada",
            "alternativa **a**"
            ]
        b_filters = [
            "la respuesta correcta es b)", "la respuesta correcta es: b)", "puedo concluir que la opcion b) es la inferencia consistente",
            "respuesta correcta es: b)", "la hipótesis que guio el experimento es: b)", "el objetivo de investigación pertinente es: b)",
            "respuesta correcta es la opcion b)", "la opcion correcta es: b)", "la respuesta más probable es la b)",
            "la opcion b) es la inferencia consistente", "la opcion b) es la que permite comprobar la relación",
            "la pregunta que se ajusta a este experimento es: b)", "la respuesta más precisa es: b)",
            "lo que se puede hacer mediante la opcion b).",
            "la respuesta mas precisa es b)",
            ]
        c_filters = [
            "la respuesta correcta es c)", "la respuesta correcta es: c)", "puedo concluir que la opcion c) es la inferencia consistente",
            "respuesta correcta es: c)", "la hipótesis que guio el experimento es: c)", "el objetivo de investigación pertinente es: c)",
            "respuesta correcta es la opcion c)", "la opcion correcta es: c)", "la respuesta más probable es la c)",
            "la opcion c) es la inferencia consistente", "la opcion c) es la que permite comprobar la relación",
            "la pregunta que se ajusta a este experimento es: c)", "la respuesta más precisa es: c)",
            "lo que se puede hacer mediante la opcion c).",
            "por lo tanto, la hipotesis que guio el experimento es c)", "el objetivo de investigacion pertinente es c)", "la opcion correcta es c)",
            "la pregunta que se ajusta a este experimento es c)", "la opcion que mejor describe el analogo del circuito hogareño es c)",
            "la opcion c) es la inferencia mas consistente",
            ]
        d_filters = [
            "la respuesta correcta es d)", "la respuesta correcta es: d)", "puedo concluir que la opcion d) es la inferencia consistente",
            "respuesta correcta es: d)", "la hipótesis que guio el experimento es: d)", "el objetivo de investigación pertinente es: d)",
            "respuesta correcta es la opcion d)", "la opcion correcta es: d)", "la respuesta más probable es la d)",
            "la opcion d) es la inferencia consistente", "la opcion d) es la que permite comprobar la relación",
            "la pregunta que se ajusta a este experimento es: d)", "la respuesta más precisa es: d)",
            "lo que se puede hacer mediante la opcion d).",
            "la respuesta mas probable es la d)", "la respuesta correcta es la d)",
            ]
        # ---------------------------------------------------------------

        # Determinar la letra correspondiente
        if self.any_filter_in_text(none_filters, text):
            letter = "0"
        elif self.any_filter_in_text(a_filters, text):
            letter = "a"
        elif self.any_filter_in_text(b_filters, text):
            letter = "b"
        elif self.any_filter_in_text(c_filters, text):
            letter = "c"
        elif self.any_filter_in_text(d_filters, text):
            letter = "d"
        
        if letter == "X":
            self.index+= 1
            print(f"--- {self.index}) ---\n",text) 

        return letter
    
    def calculate_accuracy(self, llm_answers:pd.DataFrame) -> pd.DataFrame:
        if self.respuestas_correctas is None:
            CORRECT_ANSWERS:list = [
                'd', 'c', 'c', 'b', 'b', 'b', 'a', 'a', 'b', 'd', 'c', 'd', 
                'c', 'c', 'd', 'c', 'a', 'b', 'c', 'd', 'c', 'd', 'd', 'c', 
                'd', 'b', 'd', 'a', 'c', 'd'
            ]
        else:
            CORRECT_ANSWERS = self.respuestas_correctas

        df_corrects:pd.DataFrame = pd.DataFrame(
            CORRECT_ANSWERS,
            columns= ["correct_answer"]
        )
        assert len(llm_answers) == len(df_corrects), "Dataframes are not of same lenght"

        llm_answers["correct"] = llm_answers["llm_answer"] == df_corrects["correct_answer"] #False if not equal, True otherwise
        llm_answers["correct"] = llm_answers["correct"].astype(int) #Tranforms bool to int (0 incorrect, 1 correct)

        return llm_answers
    
    def save_data_to_tesis_file(
            self, tesis_path:Path, 
            df_data:pd.DataFrame,
            file_path:Path) -> pd.DataFrame:
    
        df_tesis = pd.read_excel(tesis_path)
        if not df_tesis.columns.equals(pd.Index([
                "model", "temperature", "prompt_id", "file_id", "question_number",
                "correct", "llm_answer", "completion_tokens", "prompt_tokens", "exec_time"
                ])
            ):
            raise Exception(f"The dataframe 'tesis_results' doesnt match the desire structure\nTesis Path: {tesis_path}")

        for idx, row in df_data.iterrows():
            params_dict:dict = json.loads(row["params"])
            usage_dict:dict = json.loads(row["usage"])
            if type(params_dict) != dict:
                raise Exception("Couldnt get the dict objects from json values for 'params'")
            if type(usage_dict) != dict:
                raise Exception("Couldnt get the dict objects from json values for 'usage'")

            prompt_id:str = str(file_path.parents[0].name)
            if not "prompt" in prompt_id:
                raise Exception(f"Wrong path code,\nfile_path.parent[0]: {prompt_id}")

            if "t1" in prompt_id:
                # id_value = prompt_id.split("id_")[1].split("_")[0]
                prompt_id = prompt_id.replace("_t1","")
                prompt_id = prompt_id.replace("prompt_id_","")
            else:
                prompt_id = prompt_id.replace("prompt_id_","")

            file_name:str = str(file_path.stem)
            # print(file_name[:10])
            if file_name[:10] != "resultado_":
                raise Exception(f"Wrong file_name getted\nFile Name: {file_name}")
            file_id:int = int(file_name.split("resultado_")[1])
            
            new_record:dict = {
                "model": row["model"],
                "temperature": params_dict["temperature"],
                "prompt_id": prompt_id,
                "file_id": file_id,
                "question_number": row["question_num"],
                "correct": row["correct"],
                "llm_answer": row["llm_answer"],
                "completion_tokens": usage_dict["completion_tokens"],
                "prompt_tokens": usage_dict["prompt_tokens"],
                "exec_time": row["exec_time"],
            }
            #concat with df_tesis
            if self.guardar_registros:
                df_tesis = pd.concat(
                        [df_tesis, pd.DataFrame([new_record])], 
                        ignore_index=True
                    )
        #END FOR (iterrows)
        return df_tesis

## Codigos para seleccionar resultados y recorrerlos
En el siguiente codigo se deben elegir las carpetas que contienen los archivos excel que fueron resultados de las llamadas a la api, estas carpetas y sus archivos excel seran recorridos para obtener las respuestas del modelo

In [36]:
import tkinter as tk
from tkinter import filedialog
from pathlib import Path

def select_folders():
    root = tk.Tk()
    root.withdraw()  # Oculta la ventana principal de Tkinter
    folders = []
    while True:
        folder = filedialog.askdirectory(title="Selecciona una carpeta (Cancel para terminar)")
        if folder in folders:
            popup("THIS FOLDER IS ALREADY ADDED", "WARNING...")
        elif folder:
            folders.append(folder)
        else:
            break
    return folders

def get_xlsx_files(folders):
    xlsx_files = {}
    for folder in folders:
        folder_path = Path(folder)
        xlsx_files[folder_path.name] = list(folder_path.glob("*.xlsx"))
    return xlsx_files

popup("A continuación elija una por una las carpetas 'prompt_id' donde estan los archivos xlsx\nAl terminar de añadirlas todas, presionar 'cancelar'", "Añada las carpetas 'prompt_id'")
selected_folders = select_folders()
folders_data_llama3 = {}
if selected_folders:
    xlsx_files = get_xlsx_files(selected_folders)
    for folder, files in xlsx_files.items():
        print(f"Folder: {folder}")
        folders_data_llama3[folder] = []
        for file in files:
            print(f" - {file}")
            folders_data_llama3[folder].append(Path(file))

<h3>Este código efectivamente realiza la evaluación del rendimiento del modelo</h3>

## Codigo genera el archivo tidy
El Excel sirve para guardar los registros en formato tidy, tiene la siguiente estructura.

Columnas:
- model: str para identificar el modelo
- temperature: float de temperatura configurada
- prompt_id: int para identificar el prompt, dado por el nombre de la carpeta que contiene los archivos excel, ej "prompt_1" seria id:1
- file_ id: int para identificar el archivo, dado por su nombre, ej "resultado_2", file_id:2
- question_number: int numero de la pregunta 
- correct: 0 o 1 dependiendo de si la respuesta del modelo fue incorrecta o correcta respectivamente
- llm_answer: str alternativa dada por el modelo, puede ser "a", "b", "c", "d".
- completion_tokens: int cantidad de tokens generadas por el modelo.
- prompt_tokens: int cantidad de tokens entregados en el prompt al modelo
- exec_time: float tiempo que se demoró en terminar de responder el modelo

In [ ]:
def generate_excel_file(file_path):
    # Datos de ejemplo (puedes ajustar estos datos según tus necesidades)
    data = {
        "model": [],
        "temperature": [],
        "prompt_id": [],
        "file_id": [],
        "question_number": [],
        "correct": [],
        "llm_answer": [],
        "completion_tokens": [],
        "prompt_tokens": [],
        "exec_time": [],
    }

    # Crear un DataFrame desde los datos
    df = pd.DataFrame(data)

    # Escribir el DataFrame en un archivo Excel
    aux = os.path.join(file_path, "tidy_data.xlsx")
    if os.path.exists(aux):
        raise Exception("La ruta seleccionada ya posee un archivo 'tidy_data.xlsx'")
    else:
        df.to_excel(aux, index=False)
    del aux

path = seleccionar_carpeta(msg="Seleccione la carpeta donde desea crear el archivo xlsx listo para guardar en formato tidy")
generate_excel_file(path)


## Guardar registros formato tidy

In [ ]:
# Respuestas PAES-M1-2024-Tesis
respuestas_correctas = [
    'b', 'b', 'd', 'b', 'a', 'b', 'c', 'd', 'c', 
    'c', 'b', 'd', 'd', 'a', 'b', 'a', 'b', 'a', 
    'b', 'c', 'b', 'c', 'c', 'c', 'b', 'c', 'b',
    'd', 'c', 'd']


for key, data in folders_data_llama3.items():
    RegexAnalyzerFisica(
        files_paths=data, 
        respuestas_correctas=respuestas_correctas,
        guardar_registros=False
        )
"""
guardar_registros = False, evita que se guarden los registros, de esta forma
se puede ver en consola las preguntas que el código no logró interpretar, de esta
forma se puede modificar el codigo para registrar la alternativa correcta
"""